<a href="https://colab.research.google.com/github/cksdlakstp12/izone_finder/blob/main/IZONE_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

keras multi-gpu : https://3months.tistory.com/211

GCP : https://kim6394.tistory.com/98

function ClickConnect(){
    console.log("1분마다 코랩 연결 끊김 방지"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect-icon")
.click();
}
setInterval(ClickConnect, 1000 * 60);

In [ ]:
import zipfile
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf
import datetime
import zipfile

from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras.layers import Dense, Input, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras import layers
from keras import backend
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error : Creating directory. " + directory)

# IZ ONE Dataset Load

In [ ]:
IZ_ONE = {
    "최예나":"Choi_Yena", 
    "조유리":"Jo_Yuri",
    "김채원":"Kim_Chaewon", 
    "안유진":"Ahn_Yujin", 
    "김민주":"Kim_Minjoo", 
    "장원영":"Jang_Wonyoung",
    "나코":"Yabuki_Naco", 
    "히토미":"Honda_Hitomi", 
    "사쿠라":"Miyawaki_Sakura", 
    "이채연":"Lee_Chaeyeon",
    "강혜원":"Kang_Hyewon", 
    "권은비":"Kwon_Eunbi"
}

In [ ]:
for mode in ["train", "test"]:
  for member_name in IZ_ONE:
    createFolder(f"/content/IZ_ONE/{mode}/{member_name}") 

In [ ]:
for eng_name in IZ_ONE.values():
  IZ_ONE_train = zipfile.ZipFile(
      f'/content/drive/MyDrive/IZONE_finder/member_dataset/train/{eng_name}_train.zip')
  IZ_ONE_train.extractall(f'/content/IZ_ONE/train/{eng_name}')
  IZ_ONE_train.close()

  # IZ_ONE_test = zipfile.ZipFile(
  #     f'/content/drive/MyDrive/IZONE_finder/member_dataset/test/{eng_name}_test.zip')
  # IZ_ONE_test.extractall(f'/content/IZ_ONE/test/{eng_name}')  
  # IZ_ONE_test.close()

In [ ]:
train_img_path = "/content/IZ_ONE/train"
test_img_path = "/content/IZ_ONE/test"

member_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.05
)

train_generator = member_generator.flow_from_directory(
    train_img_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg",
    subset="training"
)

validation_generator = member_generator.flow_from_directory(
    train_img_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg",
    subset="validation"
)

test_generator = member_generator.flow_from_directory(
    test_img_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg"
)

# Gender Dataset Generate

In [ ]:
root_path = "/content/img_align_celeba/"
male_path = root_path + "male/"
female_path = root_path + "female/"

In [ ]:
createFolder(root_path)
createFolder(male_path)
createFolder(female_path)

In [ ]:
celeba = zipfile.ZipFile(f'/content/drive/MyDrive/dataset/celeba_large/img_align_celeba.zip')
celeba.extractall(root_path)

celeba.close()

In [ ]:
label_dir = "/content/drive/MyDrive/dataset/celeba_large/list_attr_celeba.txt"

with open(label_dir, "r") as f:
  file_lines = f.readlines()
  labels = []

  for file_line in file_lines[2:]:
    file_line = file_line.split(" ")

    while 1: 
      try: file_line.remove("")
      except: break
      
    name = file_line[0]
    label = int(file_line[20])
    labels.append([name, label])

In [ ]:
print(len(os.listdir(root_path + "img_align_celeba")))

202599


In [ ]:
for name, label in labels:
  name_path = root_path + "img_align_celeba/" + name
  
  selected_path = male_path if label == 1 else female_path
  
  command = f"mv {name_path} {selected_path}"
  print(command)

  os.system(command)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
mv /content/img_align_celeba/img_align_celeba/197600.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197601.jpg /content/img_align_celeba/male/
mv /content/img_align_celeba/img_align_celeba/197602.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197603.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197604.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197605.jpg /content/img_align_celeba/male/
mv /content/img_align_celeba/img_align_celeba/197606.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197607.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197608.jpg /content/img_align_celeba/female/
mv /content/img_align_celeba/img_align_celeba/197609.jpg /content/img_align_celeba/male/
mv /content/img_align_celeba/img_align_celeba/197610.jpg /co

In [ ]:
# male
with zipfile.ZipFile('/content/male.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  img_name_list = os.listdir(male_path)

  for img_name in img_name_list:
    new_zip.write(male_path + img_name, arcname=img_name) 

# female
with zipfile.ZipFile('/content/female.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  img_name_list = os.listdir(female_path)

  for img_name in img_name_list:
    new_zip.write(female_path + img_name, arcname=img_name) 

In [ ]:
sum = 0

for name in ["male", "female"]:
  for tag in ["train", "test"]:
    os.system(f"cp /content/{name}_{tag}.zip /content/drive/MyDrive/dataset/celeba_large")

  len_folder = len(os.listdir(f"/content/img_align_celeba/{name}"))
  sum += len_folder
  print(len_folder)
  print("="*80)

print("nice" if sum == 202599 else "bad")

92189
110410
nice


In [ ]:
print(len(os.listdir(f"/content/img_align_celeba/female")))

110410


In [ ]:
!ls -lah img_align_celeba

total 12M
drwxr-xr-x 5 root root 4.0K Aug 17 13:00 .
drwxr-xr-x 1 root root 4.0K Aug 17 13:57 ..
drwxr-xr-x 2 root root 3.2M Aug 17 13:56 female
drwxr-xr-x 2 root root 5.6M Aug 17 13:56 img_align_celeba
drwxr-xr-x 2 root root 2.6M Aug 17 13:56 male


split train and test

In [ ]:
train_path = "/content/celeba_gender/train/"
test_path = "/content/celeba_gender/test/"

male_train_path = train_path + "male/"
female_train_path = train_path + "female/"

male_test_path = test_path + "male/"
female_test_path = test_path + "female/"

In [ ]:
!rm -rf /content/celeba_gender/test/male
!rm -rf /content/celeba_gender/test/female

In [ ]:
createFolder(male_test_path)
createFolder(female_test_path)

In [ ]:
import random as R

for gender in ["male", "female"]:
  for file in os.listdir(train_path + gender):
    ri = R.randint(1, 100)
    if ri <= 20: os.system(f"mv {train_path}/{gender}/{file} {test_path}/{gender}/")

FileNotFoundError: ignored

In [ ]:
sum = 0
for mode in ["train", "test"]:
  for name in ["male", "female"]:
    length = len(os.listdir(f"/content/celeba_gender/{mode}/{name}"))
    sum += length
    print(length)  

print(sum)

73661
88441
18528
21969
202599


In [ ]:
total_sum = 0
for gender, mode, path in zip(["male", "female", "male", "female"], ["train", "train", "test", "test"], [male_train_path, female_train_path, male_test_path, female_test_path]):
  sum = 0
  with zipfile.ZipFile(f'/content/{gender}_{mode}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    img_name_list = os.listdir(path)

    for img_name in img_name_list:
      total_sum += 1
      sum += 1
      
      new_zip.write(path + img_name, arcname=img_name) 

    print(sum)

  os.system(f"cp /content/{gender}_{mode}.zip /content/drive/MyDrive/dataset/celeba_large/split")


print("nice" if total_sum == 202599 else "bad")

73661
88441
18528
21969
nice


In [ ]:
test_male_path = "/content/celeba_gender/test/male"
test_female_path = "/content/celeba_gender/test/female"

test_male_files = os.listdir(test_male_path)
test_female_files = os.listdir(test_female_path)

train_male_path = "/content/celeba_gender/train/male"
train_female_path = "/content/celeba_gender/train/female"

train_male_files = os.listdir(train_male_path)
train_female_files = os.listdir(train_female_path)

for m_f in test_male_files:
  os.system(f"mv {test_male_path}/{m_f} {train_male_path}")

for f_f in test_female_files:
  os.system(f"mv {test_female_path}/{f_f} {train_female_path}")

import random as R

for m_f in train_male_files:
  r = R.randint(1, 100)
  if r == 1:
    os.system(f"mv {train_male_path}/{m_f} {test_male_path}")

for f_f in train_female_files:
  r = R.randint(1, 100)
  if r == 1:
    os.system(f"mv {train_female_path}/{f_f} {test_female_path}")

print(len(os.listdir(train_male_path)))
print(len(os.listdir(train_female_path)))
print(len(os.listdir(test_male_path)))
print(len(os.listdir(test_female_path)))

91268
109306
0
0


In [ ]:
total_sum = 0
for gender, mode, path in zip(["male", "female", "male", "female"], ["train", "train", "test", "test"], [train_male_path, train_female_path, test_male_path, test_female_path]):
  sum = 0
  with zipfile.ZipFile(f'/content/{gender}_{mode}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    img_name_list = os.listdir(path)

    for img_name in img_name_list:
      total_sum += 1
      sum += 1
      
      new_zip.write(os.path.join(path, img_name), arcname=img_name) 

    print(sum)

  os.system(f"cp /content/{gender}_{mode}.zip /content/drive/MyDrive/dataset/celeba_large/split")


print("nice" if total_sum == 202599 else "bad")

90346
108239
922
1067
bad


In [ ]:
!ls -lha

total 1.4G
drwxr-xr-x 1 root root 4.0K Sep  1 11:30 .
drwxr-xr-x 1 root root 4.0K Sep  1 10:36 ..
drwxr-xr-x 4 root root 4.0K Sep  1 10:39 celeba_gender
drwxr-xr-x 4 root root 4.0K Aug 25 13:35 .config
drwx------ 6 root root 4.0K Sep  1 10:39 drive
-rw-r--r-- 1 root root 143M Sep  1 11:31 female_test.zip
-rw-r--r-- 1 root root 575M Sep  1 11:30 female_train.zip
-rw-r--r-- 1 root root 125M Sep  1 11:30 male_test.zip
-rw-r--r-- 1 root root 493M Sep  1 11:29 male_train.zip
drwxr-xr-x 1 root root 4.0K Aug 25 13:35 sample_data


# Gender Dataset Load

In [ ]:
train_path = "/content/celeba_gender/train/"
test_path = "/content/celeba_gender/test/"

male_train_path = train_path + "male"
female_train_path = train_path + "female"

male_test_path = test_path + "male"
female_test_path = test_path + "female"

In [ ]:
createFolder(male_train_path)
createFolder(female_train_path)

createFolder(male_test_path)
createFolder(female_test_path)

In [ ]:
for mode in ["train", "test"]:
  root = train_path if mode == "train" else test_path
  for gender in ["male", "female"]:
      celeba = zipfile.ZipFile(f'/content/drive/MyDrive/dataset/celeba_large/split/{gender}_{mode}.zip')
      
      celeba.extractall(root + gender)

      celeba.close()

In [ ]:
for mode in ["train", "test"]:
  root = train_path if mode == "train" else test_path
  for gender in ["male", "female"]:
    print(len(os.listdir(root + gender)))

90346
108239
922
1067


In [ ]:
gender_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.05
)

train_generator = gender_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg",
    subset="training"
)

validation_generator = gender_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg",
    subset="validation"
)

test_generator = gender_generator.flow_from_directory(
    test_path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=929,
    save_format="jpg",
)

Found 188657 images belonging to 2 classes.
Found 9928 images belonging to 2 classes.
Found 1989 images belonging to 2 classes.


# resnet

In [ ]:
# def resnet(class_num, num_layers):
#     conv_structures = [
#         # input, kernel, strides
#         [(64, (1, 1), (1, 1)), (64, (3, 3), (1, 1)), (256, (1, 1), (1, 1))],
#         [(128, (1, 1), (2, 2)), (128, (3, 3), (1, 1)), (512, (1, 1), (1, 1))],
#         [(256, (1, 1), (2, 2)), (256, (3, 3), (1, 1)), (1024, (1, 1), (1, 1))],
#         [(512, (1, 1), (2, 2)), (512, (3, 3), (1, 1)), (2048, (1, 1), (1, 1))],
#     ]

#     repeat_list = {
#                    50:[3, 4, 6, 3],
#                    101:[3, 4, 23, 3],
#                    152:[3, 8, 36, 3]
#     }
#     repeat_list = repeat_list[num_layers]
    
#     input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
#     x = ZeroPadding2D(padding=(3, 3))(input_tensor)
#     x = Conv2D(64, (7, 7), strides=(2, 2))(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = ZeroPadding2D(padding=(1,1))(x)
#     x = MaxPooling2D((3, 3), 2)(x)

#     shortcut = x        

#     for repeat, cs in zip(repeat_list, conv_structures):
#         for i in range(repeat):     
#             if(i == 0):            
#                 x = Conv2D(cs[0][0], cs[0][1], strides=cs[0][2], padding='valid')(x)
#                 x = BatchNormalization()(x)
#                 x = Activation('relu')(x)        

#                 x = Conv2D(cs[1][0], cs[1][1], strides=cs[1][2], padding='same')(x)
#                 x = BatchNormalization()(x)
#                 x = Activation('relu')(x)  

#                 x = Conv2D(cs[2][0], cs[2][1], strides=cs[2][2], padding='valid')(x)
#                 shortcut = Conv2D(cs[2][0], cs[0][1], strides=cs[0][2], padding='valid')(shortcut)
#                 x = BatchNormalization()(x)
#                 shortcut = BatchNormalization()(shortcut)

#                 x = Add()([x, shortcut]) 
#                 x = Activation('relu')(x)

#                 shortcut = x               

#             else:
#                 x = Conv2D(cs[0][0], cs[0][1], strides=cs[0][1], padding='valid')(x)
#                 x = BatchNormalization()(x)
#                 x = Activation('relu')(x)

#                 x = Conv2D(cs[1][0], cs[1][1], strides=cs[1][2], padding='same')(x)
#                 x = BatchNormalization()(x)
#                 x = Activation('relu')(x)

#                 x = Conv2D(cs[2][0], cs[2][1], strides=cs[2][2], padding='valid')(x)
#                 x = BatchNormalization()(x)            

#                 x = Add()([x, shortcut])    
#                 x = Activation('relu')(x)

#                 shortcut = x

#     x = GlobalAveragePooling2D()(x)
#     output_tensor = Dense(class_num, activation='softmax')(x)

#     resnet = Model(input_tensor, output_tensor)
    
#     resnet.summary()
    
#     return resnet

In [ ]:
# model = resnet(2, 50)

# Custom EfficientNetB4

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip('horizontal'),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.Rescaling((1./255))
])

hyperparameters

In [ ]:
num_classes = 2
model_name = "gender_classification_model"
learning_rate = 1e-5
log_dir = "logs/my_board/" + datetime.datetime.now().strftime("$Y$m$d-$H$M$S")

gender model 일 경우 사용

In [ ]:
original_dim = (72, 59, 3)
target_size = (224, 224)

input = Input(original_dim)
x = layers.Lambda(lambda image: tf.image.resize(image, target_size))(input)
x = data_augmentation(x)

gender model 아닐 경우 사용

In [ ]:
num_classes = 12
model_name = "member_classification_model"

input = Input((224, 224 ,3))
x = data_augmentation(input)

model

In [ ]:
x = EfficientNetB4(include_top=True, weights='imagenet', input_tensor=x)
x = x.output
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=input, outputs=x, name=model_name)

model.summary()

ValueError: ignored

# simple conv model + augmentation

In [ ]:
# def simple_model(class_num):
#   input = Input((224, 224 ,3))
#   x = RandomFlip("horizontal")(input)
#   x = RandomRotation((-0.1, 0.1))(x)
#   x = Conv2D(32, 3, activation="relu")(x)
#   x = MaxPool2D(2)(x)
#   x = Conv2D(64, 3, activation="relu")(x)
#   x = MaxPool2D(2)(x)
#   x = Conv2D(64, 3, activation="relu")(x)
#   x = MaxPool2D(2)(x)
#   x  = Flatten()(x)
#   x = Dense(64, activation="relu")(x)
#   outputs = Dense(class_num, activation="softmax")(x)
#   model = Model(input, outputs)
#   model.summary()
  
#   return model

In [ ]:
# class_num = 12
# model = simple_model(class_num=class_num)

NameError: ignored

# multi model

In [ ]:
def multi_model():
  input = Input((224, 224 ,3))

  l1 = Conv2D(32, 3, activation="relu")(input)
  l2 = MaxPool2D(2)(l1)
  l3 = Conv2D(64, 3, activation="relu")(l2)
  l4 = MaxPool2D(2)(l3)
  l5 = Conv2D(64, 3, activation="relu")(l4)
  l6 = MaxPool2D(2)(l5)

  l7  = Flatten()(l6)
  latent_vector = Dense(64, activation="relu")(l7)

  gender_outputs = Dense(2, activation="softmax")(latent_vector)
  smile_outputs = Dense(2, activation="softmax")(latent_vector)

  model = Model(input, [gender_outputs, smile_outputs])
  return model

In [ ]:
model = multi_model()

# utils

In [ ]:
file_name = "gender_custom_efficient_net_b4_model.h5"

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# model = tf.keras.models.load_model(file_name)

In [ ]:
backend.set_value(model.optimizer.learning_rate, learning_rate)

In [ ]:
checkpoint = ModelCheckpoint(file_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto'
                            )

earlystopping = EarlyStopping(monitor='val_loss',
                              patience=3,
                              verbose=1
                             )

tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
gender_hist = model.fit_generator(
    train_generator, 
    validation_data=validation_generator, 
    epochs=50, 
    verbose=1, 
    callbacks=[checkpoint, earlystopping, tensorboard]
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
  19/4053 [..............................] - ETA: 24:04:34 - loss: 0.8581 - accuracy: 0.5099

KeyboardInterrupt: ignored

In [ ]:
!ls -lah

In [ ]:
model.evaluate_generator(test_generator, step=1000)

In [ ]:
model.save(f"/content/drive/MyDrive/IZONE_finder/IZ_ONE_Finder/weights/{file_name}")

In [ ]:
plt.plot(gender_hist.history["accuracy"], label = "gender_accuracy")
plt.plot(gender_hist.history["loss"], label = "gender_loss")
plt.plot(gender_hist.history["val_accuracy"], label = "gender_val_accuracy")
plt.plot(gender_hist.history["val_loss"], label = "gender_val_loss")

plt.legend(loc="upper left")
plt.show()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {log_dir}

In [ ]:
# model = tf.keras.models.load_model("gender_model_simple_model.h5")
# # model = tf.keras.models.load_model("model.h5")
# model.evaluate(test_images, test_gender_labels)

In [ ]:
path = "/content/drive/MyDrive/IZONE_finder/tmp/Choi_Yena_faces/"
imgs = os.listdir(path)

for img_name in imgs:
  img = np.asarray(Image.open(path + img_name)) / 255
  img = img.reshape(1, 224, 224, 3)
  predict = model.predict(img)
  print(predict)
  print("여자" if predict[0][0] > predict[0][1] else "남자")
  plt.imshow(img[0])
  plt.show()

In [ ]:
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>